# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [9]:
symbol = 'aapl'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [13]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

13.18

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [18]:
df = pd.DataFrame(columns = my_columns)
for batch in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in batch.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index=my_columns
            ),
            ignore_index=True
        )
df.dropna(inplace=True)
df


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,164.48,50.16,N/A
1,AAL,21.48,-4.33,N/A
2,AAP,250.03,25.93,N/A
3,AAPL,151.12,13.51,N/A
4,ABBV,122.02,28.86,N/A
...,...,...,...,...
500,YUM,128.91,24.54,N/A
501,ZBH,137.42,34.07,N/A
502,ZBRA,605.83,37.75,N/A
503,ZION,66.85,6.46,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [20]:
df.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)
df =  df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True, drop=True)
df

/var/folders/q_/gmxdkf893w3bm9wxvh6635t80000gp/T/ipykernel_89390/1321168316.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.sort_values('Price-to-Earnings Ratio', ascending=False, inplace=True)


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LUV,50.48,15000.4,N/A
1,HLT,149.34,1059.6,N/A
2,NOW,710.62,635.68,N/A
3,ALK,58.19,562.41,N/A
4,FIS,122.42,335.12,N/A
5,UDR,56.41,303.36,N/A
6,KSU,313.85,299.58,N/A
7,BKNG,2535.62,276.86,N/A
8,DIS,165.82,263,N/A
9,T,25.08,192.93,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [25]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [26]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [29]:
position_size = portfolio_size/len(df.index)
for row in df.index:
    df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[row, 'Price'])
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,LUV,50.48,15000.4,3961
1,HLT,149.34,1059.6,1339
2,NOW,710.62,635.68,281
3,ALK,58.19,562.41,3437
4,FIS,122.42,335.12,1633
5,UDR,56.41,303.36,3545
6,KSU,313.85,299.58,637
7,BKNG,2535.62,276.86,78
8,DIS,165.82,263,1206
9,T,25.08,192.93,7974


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [35]:
symbol = 'AAPL'
batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
data = requests.get(batch_api_call_url).json()
# * Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']
# * Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
# * Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
gross_profit = data[symbol]['advanced-stats']['grossProfit']

# * Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
ev_to_ebitda = enterprise_value/ebitda

# * Enterprise Value divided by Gross Profit (EV/GP)
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [48]:
rv_columns = [
    'Ticker',
    'Price', 
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile', 
    'Price-to-Book Ratio', 
    'PB Percentile', 
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]
rv_df = pd.DataFrame(columns=rv_columns)

for batch in symbol_strings:
    batch_api_call_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json()
    for symbol in batch.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try: 
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try: 
            ev_to_gross_profit = enterprise_value/gross_profit 
        except TypeError:
            ev_to_gross_profit = np.NaN
        #if(not enterprise_value or not ebitda or not gross_profit):
            #continue
        rv_df = rv_df.append(
            pd.Series(
                [
                    symbol, 
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'], 
                    'N/A', 
                    data[symbol]['advanced-stats']['priceToBook'], 
                    'N/A', 
                    data[symbol]['advanced-stats']['priceToSales'], 
                    'N/A',
                    ev_to_ebitda,
                    'N/A', 
                    ev_to_gross_profit,
                    'N/A', 
                    'N/A'
                ],
                index=rv_columns
            ),
            ignore_index=True
        )
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.830,N/A,51.88,N/A,10.1,N/A,7.94,N/A,32.148711,N/A,15.457827,N/A,N/A
1,AAL,21.200,N/A,-4.24,N/A,-1.87,N/A,0.5692,N/A,-7.938004,N/A,1.554491,N/A,N/A
2,AAP,250.670,N/A,26,N/A,4.69,N/A,1.38,N/A,13.222151,N/A,3.111496,N/A,N/A
3,AAPL,153.176,N/A,13.41,N/A,19.55,N/A,3.44,N/A,10.346964,N/A,8.148347,N/A,N/A
4,ABBV,122.680,N/A,27.99,N/A,15.65,N/A,3.76,N/A,10.473259,N/A,7.451282,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.700,N/A,24.7,N/A,-4.85,N/A,5.82,N/A,20.962793,N/A,9.898805,N/A,N/A
501,ZBH,138.460,N/A,35.13,N/A,2.21,N/A,3.66,N/A,19.023040,N/A,6.156183,N/A,N/A
502,ZBRA,619.080,N/A,38.95,N/A,11.2,N/A,5.74,N/A,28.926954,N/A,12.373685,N/A,N/A
503,ZION,68.160,N/A,6.48,N/A,1.43,N/A,3.02,N/A,4.904040,N/A,2.697111,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [49]:
rv_df[rv_df.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
5,ABC,130.860,N/A,None,N/A,697.54,N/A,0.1281,N/A,9.769055,N/A,5.006918,N/A,N/A
28,ALXN,183.600,N/A,60.42,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,302.400,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.140,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.95,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,28.550,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,49.330,N/A,14.53,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,58.070,N/A,33.2,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,39.830,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,3033.690,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [52]:
for column in [
 'Price',
 'Price-to-Earnings Ratio',
 'Price-to-Book Ratio',
 'Price-to-Sales Ratio',
 'EV/EBITDA',
 'EV/GP']:
    rv_df[column].fillna(rv_df[column].mean(), inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.830,N/A,51.88,N/A,10.10,N/A,7.9400,N/A,32.148711,N/A,15.457827,N/A,N/A
1,AAL,21.200,N/A,-4.24,N/A,-1.87,N/A,0.5692,N/A,-7.938004,N/A,1.554491,N/A,N/A
2,AAP,250.670,N/A,26.00,N/A,4.69,N/A,1.3800,N/A,13.222151,N/A,3.111496,N/A,N/A
3,AAPL,153.176,N/A,13.41,N/A,19.55,N/A,3.4400,N/A,10.346964,N/A,8.148347,N/A,N/A
4,ABBV,122.680,N/A,27.99,N/A,15.65,N/A,3.7600,N/A,10.473259,N/A,7.451282,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.700,N/A,24.70,N/A,-4.85,N/A,5.8200,N/A,20.962793,N/A,9.898805,N/A,N/A
501,ZBH,138.460,N/A,35.13,N/A,2.21,N/A,3.6600,N/A,19.023040,N/A,6.156183,N/A,N/A
502,ZBRA,619.080,N/A,38.95,N/A,11.20,N/A,5.7400,N/A,28.926954,N/A,12.373685,N/A,N/A
503,ZION,68.160,N/A,6.48,N/A,1.43,N/A,3.0200,N/A,4.904040,N/A,2.697111,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [53]:
rv_df[rv_df.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [54]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile', 
    'Price-to-Book Ratio' :'PB Percentile', 
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for key, value in metrics.items():
    for row in rv_df.index:
        rv_df.loc[row, value] = stats.percentileofscore(rv_df[key], rv_df.loc[row,key])/100
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.830,N/A,51.88,0.829703,10.10,0.809901,7.9400,0.815842,32.148711,0.857426,15.457827,0.843564,N/A
1,AAL,21.200,N/A,-4.24,0.0534653,-1.87,0.0554455,0.5692,0.0435644,-7.938004,0.0237624,1.554491,0.0653465,N/A
2,AAP,250.670,N/A,26.00,0.508911,4.69,0.577228,1.3800,0.185149,13.222151,0.413861,3.111496,0.156436,N/A
3,AAPL,153.176,N/A,13.41,0.235644,19.55,0.924752,3.4400,0.506931,10.346964,0.279208,8.148347,0.562376,N/A
4,ABBV,122.680,N/A,27.99,0.548515,15.65,0.89901,3.7600,0.550495,10.473259,0.287129,7.451282,0.510891,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.700,N/A,24.70,0.473267,-4.85,0.0514851,5.8200,0.728713,20.962793,0.683168,9.898805,0.669307,N/A
501,ZBH,138.460,N/A,35.13,0.671287,2.21,0.30396,3.6600,0.534653,19.023040,0.629703,6.156183,0.384158,N/A
502,ZBRA,619.080,N/A,38.95,0.710891,11.20,0.831683,5.7400,0.723762,28.926954,0.815842,12.373685,0.768317,N/A
503,ZION,68.160,N/A,6.48,0.0792079,1.43,0.144554,3.0200,0.453465,4.904040,0.0534653,2.697111,0.122772,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [55]:
from statistics import mean 

for row in rv_df.index:
    value_percentiles = []
    for value in metrics.values():
        value_percentiles.append(rv_df.loc[row, value])
    rv_df.loc[row, 'RV Score'] = mean(value_percentiles)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,163.830,N/A,51.88,0.829703,10.10,0.809901,7.9400,0.815842,32.148711,0.857426,15.457827,0.843564,0.831287
1,AAL,21.200,N/A,-4.24,0.0534653,-1.87,0.0554455,0.5692,0.0435644,-7.938004,0.0237624,1.554491,0.0653465,0.0483168
2,AAP,250.670,N/A,26.00,0.508911,4.69,0.577228,1.3800,0.185149,13.222151,0.413861,3.111496,0.156436,0.368317
3,AAPL,153.176,N/A,13.41,0.235644,19.55,0.924752,3.4400,0.506931,10.346964,0.279208,8.148347,0.562376,0.501782
4,ABBV,122.680,N/A,27.99,0.548515,15.65,0.89901,3.7600,0.550495,10.473259,0.287129,7.451282,0.510891,0.559208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,129.700,N/A,24.70,0.473267,-4.85,0.0514851,5.8200,0.728713,20.962793,0.683168,9.898805,0.669307,0.521188
501,ZBH,138.460,N/A,35.13,0.671287,2.21,0.30396,3.6600,0.534653,19.023040,0.629703,6.156183,0.384158,0.504752
502,ZBRA,619.080,N/A,38.95,0.710891,11.20,0.831683,5.7400,0.723762,28.926954,0.815842,12.373685,0.768317,0.770099
503,ZION,68.160,N/A,6.48,0.0792079,1.43,0.144554,3.0200,0.453465,4.904040,0.0534653,2.697111,0.122772,0.170693


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [56]:
rv_df.sort_values('RV Score', ascending=True, inplace=True)
rv_df = rv_df[:50]
rv_df.reset_index(drop = True, inplace=True)
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.510,N/A,6.83,0.0831683,0.4897,0.0574257,0.4157,0.0257426,3.218049,0.0316832,0.392112,0.00792079,0.0411881
1,AAL,21.200,N/A,-4.24,0.0534653,-1.8700,0.0554455,0.5692,0.0435644,-7.938004,0.0237624,1.554491,0.0653465,0.0483168
2,PRU,111.450,N/A,6.23,0.0772277,0.6794,0.0633663,0.6106,0.049505,4.579512,0.0475248,0.586922,0.0118812,0.049901
3,AIG,60.530,N/A,9.35,0.124752,0.7658,0.0673267,1.0500,0.125743,4.517926,0.0455446,1.011481,0.029703,0.0786139
4,MET,66.710,N/A,11.05,0.168317,0.7953,0.0712871,0.7678,0.0732673,5.226619,0.0633663,0.751390,0.0178218,0.0788119
5,ALL,119.050,N/A,11.04,0.166337,1.3700,0.128713,0.6846,0.0594059,2.252298,0.029703,0.644960,0.0138614,0.079604
6,L,56.949,N/A,9.29,0.120792,0.8290,0.0752475,1.0185,0.112871,5.412637,0.0653465,0.997501,0.0277228,0.080396
7,MCK,236.150,N/A,-7.39,0.049505,-60.4200,0.0237624,0.1371,0.00792079,8.494146,0.20396,2.931066,0.136634,0.0843564
8,BA,227.400,N/A,-15.80,0.0435644,-9.1400,0.0475248,2.1200,0.323762,-35.719170,0.00594059,-1455.624041,0.0019802,0.0845545
9,AIZ,160.920,N/A,7.38,0.0970297,1.6600,0.189109,0.9202,0.0970297,1.966213,0.0277228,0.883712,0.0217822,0.0865347


In [64]:
portfolio_input()
position_size = portfolio_size/len(rv_df.index)
for row in rv_df.index:
    rv_df.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_df.loc[row, 'Price'])
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,26.510,7544,6.83,0.0831683,0.4897,0.0574257,0.4157,0.0257426,3.218049,0.0316832,0.392112,0.00792079,0.0411881
1,AAL,21.200,9433,-4.24,0.0534653,-1.8700,0.0554455,0.5692,0.0435644,-7.938004,0.0237624,1.554491,0.0653465,0.0483168
2,PRU,111.450,1794,6.23,0.0772277,0.6794,0.0633663,0.6106,0.049505,4.579512,0.0475248,0.586922,0.0118812,0.049901
3,AIG,60.530,3304,9.35,0.124752,0.7658,0.0673267,1.0500,0.125743,4.517926,0.0455446,1.011481,0.029703,0.0786139
4,MET,66.710,2998,11.05,0.168317,0.7953,0.0712871,0.7678,0.0732673,5.226619,0.0633663,0.751390,0.0178218,0.0788119
5,ALL,119.050,1679,11.04,0.166337,1.3700,0.128713,0.6846,0.0594059,2.252298,0.029703,0.644960,0.0138614,0.079604
6,L,56.949,3511,9.29,0.120792,0.8290,0.0752475,1.0185,0.112871,5.412637,0.0653465,0.997501,0.0277228,0.080396
7,MCK,236.150,846,-7.39,0.049505,-60.4200,0.0237624,0.1371,0.00792079,8.494146,0.20396,2.931066,0.136634,0.0843564
8,BA,227.400,879,-15.80,0.0435644,-9.1400,0.0475248,2.1200,0.323762,-35.719170,0.00594059,-1455.624041,0.0019802,0.0845545
9,AIZ,160.920,1242,7.38,0.0970297,1.6600,0.189109,0.9202,0.0970297,1.966213,0.0277228,0.883712,0.0217822,0.0865347


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [65]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_df.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [66]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [67]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [68]:
writer.save()